In [ ]:
import json, pandas as pd
from pandas.io.json import json_normalize

#Clean Data
dfAllSides = pd.read_json('AllSidesSources.json')
dfAllSidesTrans = dfAllSides.T
dfAllSidesTrans.index = dfAllSidesTrans.index.str.lower()
dfAllSidesTrans['rating'] = dfAllSidesTrans['rating'].str.lower()
dfAllSidesCleaned = dfAllSidesTrans.sort_index()

dfAllSidesCleaned

In [ ]:
#Figure out possible values
set(dfAllSidesCleaned.rating)

#{'center', 'lean left', 'lean right', 'left', 'mixed', 'right'}

In [ ]:
#Clean Data
dfMBFC = pd.read_json('MBFCsourcesAll.json')
dfMBFCTrans = dfMBFC.T
dfMBFCTrans.index = dfMBFCTrans.index.str.lower()
dfMBFCSorted = dfMBFCTrans.sort_index()
dfMBFCCleaned = dfMBFCSorted[dfMBFCSorted.bias != '']
dfMBFCNorm = dfMBFCCleaned[['name','bias','reporting','notes']]
dfMBFCNorm

In [ ]:
#Figure out possible values
set(dfMBFCNorm.bias)
#{center', 'conspiracy', 'fake-news', 'left', 'left-center', 'pro-science', 'right', 'right-center', 'satire'}

In [ ]:
#Normalize data, drop 'mixed' sites that give perspectives from both sides of the policial spectrum
dfAllSidesNorm = dfAllSidesCleaned.replace(['lean left', 'lean right'], ['left-center', 'right-center'])
dfAllSidesNorm = dfAllSidesNorm.rename(index=str, columns={"rating": "bias", "title": "name"})
dfAllSidesNorm = dfAllSidesNorm[dfAllSidesNorm.bias != 'mixed']
dfAllSidesNorm

In [ ]:
#get total ratings for AllSides
dfAllSidesNorm.shape

In [ ]:
#get total ratings for AllSides
dfMBFCNorm.shape

In [ ]:
#get number of agreed-upon ratings
dfMerged = pd.merge(dfAllSidesNorm, dfMBFCNorm, how='inner')
dfMerged.shape